In [1]:
from __future__ import division
import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import pearsonr, spearmanr, probplot
%matplotlib inline

## 1) Loading the Data

In [2]:
print("Load the training, test and store data using pandas")
train = pd.read_csv("input/train.csv")
test = pd.read_csv("input/test.csv")
store = pd.read_csv("input/store.csv")
states = pd.read_csv("../../data/store_states.csv")  # stores per state from https://www.kaggle.com/c/rossmann-store-sales/forums/t/17048/putting-stores-on-the-map/

Load the training, test and store data using pandas


/Users/2015/anaconda/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2902: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
train.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,1,1,0,1
1,2,5,2015-07-31,6064,625,1,1,0,1
2,3,5,2015-07-31,8314,821,1,1,0,1
3,4,5,2015-07-31,13995,1498,1,1,0,1
4,5,5,2015-07-31,4822,559,1,1,0,1


In [4]:
test.head()

,Id,Store,DayOfWeek,Date,Open,Promo,StateHoliday,SchoolHoliday
0,1,1,4,2015-09-17,1,1,0,0
1,2,3,4,2015-09-17,1,1,0,0
2,3,7,4,2015-09-17,1,1,0,0
3,4,8,4,2015-09-17,1,1,0,0
4,5,9,4,2015-09-17,1,1,0,0


In [5]:
store.head()

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,c,a,1270,9,2008,0,NaN,NaN,NaN
1,2,a,a,570,11,2007,1,13,2010,"Jan,Apr,Jul,Oct"
2,3,a,a,14130,12,2006,1,14,2011,"Jan,Apr,Jul,Oct"
3,4,c,c,620,9,2009,0,NaN,NaN,NaN
4,5,a,a,29910,4,2015,0,NaN,NaN,NaN


In [6]:
states.head()

,Store,State
0,1,HE
1,2,TH
2,3,NW
3,4,BE
4,5,SN


# Exploring the data

In [7]:
train.describe()

,Store,DayOfWeek,Sales,Customers,Open,Promo,SchoolHoliday
count,1017209.000000,1017209.000000,1017209.000000,1017209.000000,1017209.000000,1017209.000000,1017209.000000
mean,558.429727,3.998341,5773.818972,633.145946,0.830107,0.381515,0.178647
std,321.908651,1.997391,3849.926175,464.411734,0.375539,0.485759,0.383056
min,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,280.000000,2.000000,3727.000000,405.000000,1.000000,0.000000,0.000000
50%,558.000000,4.000000,5744.000000,609.000000,1.000000,0.000000,0.000000
75%,838.000000,6.000000,7856.000000,837.000000,1.000000,1.000000,0.000000
max,1115.000000,7.000000,41551.000000,7388.000000,1.000000,1.000000,1.000000


In [8]:
test.describe()

,Id,Store,DayOfWeek,Open,Promo,SchoolHoliday
count,41088.000000,41088.000000,41088.000000,41077.000000,41088.000000,41088.000000
mean,20544.500000,555.899533,3.979167,0.854322,0.395833,0.443487
std,11861.228267,320.274496,2.015481,0.352787,0.489035,0.496802
min,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000
25%,10272.750000,279.750000,2.000000,1.000000,0.000000,0.000000
50%,20544.500000,553.500000,4.000000,1.000000,0.000000,0.000000
75%,30816.250000,832.250000,6.000000,1.000000,1.000000,1.000000
max,41088.000000,1115.000000,7.000000,1.000000,1.000000,1.000000


In [9]:
store.describe()

,Store,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear
count,1115.00000,1112.000000,761.000000,761.000000,1115.000000,571.000000,571.000000
mean,558.00000,5404.901079,7.224704,2008.668857,0.512108,23.595447,2011.763573
std,322.01708,7663.174720,3.212348,6.195983,0.500078,14.141984,1.674935
min,1.00000,20.000000,1.000000,1900.000000,0.000000,1.000000,2009.000000
25%,279.50000,717.500000,4.000000,2006.000000,0.000000,13.000000,2011.000000
50%,558.00000,2325.000000,8.000000,2010.000000,1.000000,22.000000,2012.000000
75%,836.50000,6882.500000,10.000000,2013.000000,1.000000,37.000000,2013.000000
max,1115.00000,75860.000000,12.000000,2015.000000,1.000000,50.000000,2015.000000


# Missing Values

In [10]:
pd.isnull(train).any()

Store            False
DayOfWeek        False
Date             False
Sales            False
Customers        False
Open             False
Promo            False
StateHoliday     False
SchoolHoliday    False
dtype: bool

In [11]:
pd.isnull(test).any()

Id               False
Store            False
DayOfWeek        False
Date             False
Open              True
Promo            False
StateHoliday     False
SchoolHoliday    False
dtype: bool

In [12]:
pd.isnull(store).any()

Store                        False
StoreType                    False
Assortment                   False
CompetitionDistance           True
CompetitionOpenSinceMonth     True
CompetitionOpenSinceYear      True
Promo2                       False
Promo2SinceWeek               True
Promo2SinceYear               True
PromoInterval                 True
dtype: bool

In [13]:
pd.isnull(states).any()

Store    False
State    False
dtype: bool

In [14]:
test[test.isnull().any(axis=1)]

,Id,Store,DayOfWeek,Date,Open,Promo,StateHoliday,SchoolHoliday
479,480,622,4,2015-09-17,NaN,1,0,0
1335,1336,622,3,2015-09-16,NaN,1,0,0
2191,2192,622,2,2015-09-15,NaN,1,0,0
3047,3048,622,1,2015-09-14,NaN,1,0,0
4759,4760,622,6,2015-09-12,NaN,0,0,0
5615,5616,622,5,2015-09-11,NaN,0,0,0
6471,6472,622,4,2015-09-10,NaN,0,0,0
7327,7328,622,3,2015-09-09,NaN,0,0,0
8183,8184,622,2,2015-09-08,NaN,0,0,0
9039,9040,622,1,2015-09-07,NaN,0,0,0


**Only store 622 has NaN for Open feature in the test set**

In [15]:
test.loc[test['Store'] == 622]

,Id,Store,DayOfWeek,Date,Open,Promo,StateHoliday,SchoolHoliday
479,480,622,4,2015-09-17,NaN,1,0,0
1335,1336,622,3,2015-09-16,NaN,1,0,0
2191,2192,622,2,2015-09-15,NaN,1,0,0
3047,3048,622,1,2015-09-14,NaN,1,0,0
3903,3904,622,7,2015-09-13,0,0,0,0
4759,4760,622,6,2015-09-12,NaN,0,0,0
5615,5616,622,5,2015-09-11,NaN,0,0,0
6471,6472,622,4,2015-09-10,NaN,0,0,0
7327,7328,622,3,2015-09-09,NaN,0,0,0
8183,8184,622,2,2015-09-08,NaN,0,0,0


Store 622 has 11 missing values in the Open columns, but not all of the data in that column of that store is missing. Based upon the previous dates, Store 622 is closed on Sundays (Open = 0) and the rest of the dates fall on Monday to Saturday, so can impute Open = 1.

In [16]:
# Impute Open = 1 for store 622
test['Open'].fillna(1, inplace=True)

# Separating the Date column into Features

In [17]:
from numpy import double
#Separating the date columns
train['year'] = train.Date.apply(lambda x: x.split('-')[0])
train['year'] = train['year'].astype(double)
train['month'] = train.Date.apply(lambda x: x.split('-')[1])
train['month'] = train['month'].astype(double)
train['day'] = train.Date.apply(lambda x: x.split('-')[2])
train['day'] = train['day'].astype(double)

In [18]:
train.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,year,month,day
0,1,5,2015-07-31,5263,555,1,1,0,1,2015,7,31
1,2,5,2015-07-31,6064,625,1,1,0,1,2015,7,31
2,3,5,2015-07-31,8314,821,1,1,0,1,2015,7,31
3,4,5,2015-07-31,13995,1498,1,1,0,1,2015,7,31
4,5,5,2015-07-31,4822,559,1,1,0,1,2015,7,31


# Add additional Date columns

* day_of_year
* weekofyear
* quarter
* is_month_start
* is_month_end
* is_quarter_start
* is_quarter_end

In [19]:
# create test data frame first
df = train

In [20]:
df['Date'] = pd.to_datetime(df['Date'])

In [21]:
import datetime
df['day_of_year'] = df['Date'].dt.dayofyear
df['weekofyear'] = df['Date'].dt.weekofyear
df['quarter'] = df['Date'].dt.quarter
df['is_month_start'] = df['Date'].dt.is_month_start.astype(int)
df['is_month_end'] = df['Date'].dt.is_month_end.astype(int)
df['is_quarter_start'] = df['Date'].dt.is_quarter_start.astype(int)
df['is_quarter_end'] = df['Date'].dt.is_quarter_end.astype(int)

In [22]:
df.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,year,month,day,day_of_year,weekofyear,quarter,is_month_start,is_month_end,is_quarter_start,is_quarter_end
0,1,5,2015-07-31,5263,555,1,1,0,1,2015,7,31,212,31,3,0,1,0,0
1,2,5,2015-07-31,6064,625,1,1,0,1,2015,7,31,212,31,3,0,1,0,0
2,3,5,2015-07-31,8314,821,1,1,0,1,2015,7,31,212,31,3,0,1,0,0
3,4,5,2015-07-31,13995,1498,1,1,0,1,2015,7,31,212,31,3,0,1,0,0
4,5,5,2015-07-31,4822,559,1,1,0,1,2015,7,31,212,31,3,0,1,0,0


In [23]:
df.loc[(df['Open'] == 1) & ((df['Sales'] == 0) | (df['Customers'] == 0))].shape

(54, 19)

**54 stores have a day when they are listed as Open but no Sales and/or No customers. It sppears that this is the day before the re-open after being refurbished. Can safely be set to 0**

In [24]:
df.ix[(df['Open'] == 1) & ((df['Sales'] == 0) | (df['Customers'] == 0))] = 0

In [25]:
# confirm that Open has been set to 0 for stores with Open=1 and no sales and no customers
df.loc[(df['Open'] == 1) & ((df['Sales'] == 0) | (df['Customers'] == 0))].shape

(0, 19)

In [26]:
# now add date features to train data set
train['Date'] = pd.to_datetime(train['Date'])

In [27]:
train['day_of_year'] = train['Date'].dt.dayofyear
train['weekofyear'] = train['Date'].dt.weekofyear
train['quarter'] = train['Date'].dt.quarter
train['is_month_start'] = train['Date'].dt.is_month_start.astype(int)
train['is_month_end'] = train['Date'].dt.is_month_end.astype(int)
train['is_quarter_start'] = train['Date'].dt.is_quarter_start.astype(int)
train['is_quarter_end'] = train['Date'].dt.is_quarter_end.astype(int)

In [28]:
train.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,year,month,day,day_of_year,weekofyear,quarter,is_month_start,is_month_end,is_quarter_start,is_quarter_end
0,1,5,2015-07-31,5263,555,1,1,0,1,2015,7,31,212,31,3,0,1,0,0
1,2,5,2015-07-31,6064,625,1,1,0,1,2015,7,31,212,31,3,0,1,0,0
2,3,5,2015-07-31,8314,821,1,1,0,1,2015,7,31,212,31,3,0,1,0,0
3,4,5,2015-07-31,13995,1498,1,1,0,1,2015,7,31,212,31,3,0,1,0,0
4,5,5,2015-07-31,4822,559,1,1,0,1,2015,7,31,212,31,3,0,1,0,0


In [29]:
# check how many stores are listed as Open but have 0 sales and/or 0 customers
train.loc[(train['Open'] == 1) & ((train['Sales'] == 0) | (train['Customers'] == 0))].shape

(0, 19)

In [30]:
train.ix[(train['Open'] == 1) & ((train['Sales'] == 0) | (train['Customers'] == 0))] = 0

In [31]:
# check how many stores are listed as Open but have 0 sales and/or 0 customers
train.loc[(train['Open'] == 1) & ((train['Sales'] == 0) | (train['Customers'] == 0))].shape

(0, 19)

# State Holidays

In [32]:
df.loc[(df['Store'] == 674) & (df['Open'] == 0) & (df['StateHoliday'] == 'b')].sort(['Store', 'Date'])

/Users/2015/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,year,month,day,day_of_year,weekofyear,quarter,is_month_start,is_month_end,is_quarter_start,is_quarter_end
919763,674,5,2013-03-29,0,0,0,1,b,1,2013,3,29,88,13,1,0,0,0,0
916418,674,1,2013-04-01,0,0,0,0,b,1,2013,4,1,91,14,2,1,0,1,0
490488,674,5,2014-04-18,0,0,0,1,b,1,2014,4,18,108,16,2,0,0,0,0
487143,674,1,2014-04-21,0,0,0,0,b,1,2014,4,21,111,17,2,0,0,0,0
133358,674,5,2015-04-03,0,0,0,1,b,1,2015,4,3,93,14,2,0,0,0,0
130013,674,1,2015-04-06,0,0,0,0,b,1,2015,4,6,96,15,2,0,0,0,0


* 'a' seems to be a pretty general category for State Holiday
* 'b' looks to be Easter. A cursory glance doesn't reveal any spike in spending but need to confirm
* 'c' is Christmas. What about Days before Christmas column?
* Is O being used as a string and an integer

In [33]:
pd.unique(df.StateHoliday.ravel())

array(['0', 'a', 0, 'b', 'c'], dtype=object)

**Yes, it is being treated as both an integer and string. Need to convert**

In [34]:
# Convert on test data frame first
df.ix[(df['StateHoliday'] == 0)] = '0'

In [35]:
pd.unique(df.StateHoliday.ravel())

array(['0', 'a', 'b', 'c'], dtype=object)

In [36]:
# Convert on train data frame
train.ix[(train['StateHoliday'] == 0)] = '0'

# Add First Day of Promo feature

In [37]:
# Test on df data frame first
df['PromoFirstDate'] = 0

In [38]:
df.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,year,month,day,day_of_year,weekofyear,quarter,is_month_start,is_month_end,is_quarter_start,is_quarter_end,PromoFirstDate
0,1,5,2015-07-31,5263,555,1,1,0,1,2015,7,31,212,31,3,0,1,0,0,0
1,2,5,2015-07-31,6064,625,1,1,0,1,2015,7,31,212,31,3,0,1,0,0,0
2,3,5,2015-07-31,8314,821,1,1,0,1,2015,7,31,212,31,3,0,1,0,0,0
3,4,5,2015-07-31,13995,1498,1,1,0,1,2015,7,31,212,31,3,0,1,0,0,0
4,5,5,2015-07-31,4822,559,1,1,0,1,2015,7,31,212,31,3,0,1,0,0,0


## Need to sort data frame first (by store and date)

In [39]:
# Test on df data frame first
df = df.sort_values(by=['Store', 'Date'], ascending=True,na_position='last')

In [47]:
df.head(15)

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,year,month,day,day_of_year,weekofyear,quarter,is_month_start,is_month_end,is_quarter_start,is_quarter_end,PromoFirstDate
1016095,1,2,2013-01-01,0,0,0,0,a,1,2013,1,1,1,1,1,1,0,1,0,0
1014980,1,3,2013-01-02,5530,668,1,0,0,1,2013,1,2,2,1,1,0,0,0,0,0
1013865,1,4,2013-01-03,4327,578,1,0,0,1,2013,1,3,3,1,1,0,0,0,0,0
1012750,1,5,2013-01-04,4486,619,1,0,0,1,2013,1,4,4,1,1,0,0,0,0,0
1011635,1,6,2013-01-05,4997,635,1,0,0,1,2013,1,5,5,1,1,0,0,0,0,0
1010520,1,7,2013-01-06,0,0,0,0,0,1,2013,1,6,6,1,1,0,0,0,0,0
1009405,1,1,2013-01-07,7176,785,1,1,0,1,2013,1,7,7,2,1,0,0,0,0,1
1008290,1,2,2013-01-08,5580,654,1,1,0,1,2013,1,8,8,2,1,0,0,0,0,0
1007175,1,3,2013-01-09,5471,626,1,1,0,1,2013,1,9,9,2,1,0,0,0,0,0
1006060,1,4,2013-01-10,4892,615,1,1,0,1,2013,1,10,10,2,1,0,0,0,0,0


**Set first day of Promo**

In [41]:
# set PromoFirstDate = 1 if Promo = 1 for current row and Promo = 0 for preceding row
df.ix[(df['Promo'] == 1 ) & (df['Promo'].shift(1) ==0), 'PromoFirstDate'] = 1

In [44]:
# glance at data frame to see if this worked correctly
df.head(10)

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,year,month,day,day_of_year,weekofyear,quarter,is_month_start,is_month_end,is_quarter_start,is_quarter_end,PromoFirstDate
1016095,1,2,2013-01-01,0,0,0,0,a,1,2013,1,1,1,1,1,1,0,1,0,0
1014980,1,3,2013-01-02,5530,668,1,0,0,1,2013,1,2,2,1,1,0,0,0,0,0
1013865,1,4,2013-01-03,4327,578,1,0,0,1,2013,1,3,3,1,1,0,0,0,0,0
1012750,1,5,2013-01-04,4486,619,1,0,0,1,2013,1,4,4,1,1,0,0,0,0,0
1011635,1,6,2013-01-05,4997,635,1,0,0,1,2013,1,5,5,1,1,0,0,0,0,0
1010520,1,7,2013-01-06,0,0,0,0,0,1,2013,1,6,6,1,1,0,0,0,0,0
1009405,1,1,2013-01-07,7176,785,1,1,0,1,2013,1,7,7,2,1,0,0,0,0,1
1008290,1,2,2013-01-08,5580,654,1,1,0,1,2013,1,8,8,2,1,0,0,0,0,0
1007175,1,3,2013-01-09,5471,626,1,1,0,1,2013,1,9,9,2,1,0,0,0,0,0
1006060,1,4,2013-01-10,4892,615,1,1,0,1,2013,1,10,10,2,1,0,0,0,0,0


In [45]:
# Now set PromoFirstDate = 1 for Train data frame
# first sort the data frame
train = train.sort_values(by=['Store', 'Date'], ascending=True,na_position='last')

In [48]:
train['PromoFirstDate'] = 0

In [49]:
# glance at data frame to see if this worked correctly
train.head(10)

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,year,month,day,day_of_year,weekofyear,quarter,is_month_start,is_month_end,is_quarter_start,is_quarter_end,PromoFirstDate
1016095,1,2,2013-01-01,0,0,0,0,a,1,2013,1,1,1,1,1,1,0,1,0,0
1014980,1,3,2013-01-02,5530,668,1,0,0,1,2013,1,2,2,1,1,0,0,0,0,0
1013865,1,4,2013-01-03,4327,578,1,0,0,1,2013,1,3,3,1,1,0,0,0,0,0
1012750,1,5,2013-01-04,4486,619,1,0,0,1,2013,1,4,4,1,1,0,0,0,0,0
1011635,1,6,2013-01-05,4997,635,1,0,0,1,2013,1,5,5,1,1,0,0,0,0,0
1010520,1,7,2013-01-06,0,0,0,0,0,1,2013,1,6,6,1,1,0,0,0,0,0
1009405,1,1,2013-01-07,7176,785,1,1,0,1,2013,1,7,7,2,1,0,0,0,0,0
1008290,1,2,2013-01-08,5580,654,1,1,0,1,2013,1,8,8,2,1,0,0,0,0,0
1007175,1,3,2013-01-09,5471,626,1,1,0,1,2013,1,9,9,2,1,0,0,0,0,0
1006060,1,4,2013-01-10,4892,615,1,1,0,1,2013,1,10,10,2,1,0,0,0,0,0


In [50]:
# set PromoFirstDate = 1 if Promo = 1 for current row and Promo = 0 for preceding row
train.ix[(train['Promo'] == 1 ) & (train['Promo'].shift(1) ==0), 'PromoFirstDate'] = 1

In [51]:
# glance at data frame to see if this worked correctly
train.head(10)

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,year,month,day,day_of_year,weekofyear,quarter,is_month_start,is_month_end,is_quarter_start,is_quarter_end,PromoFirstDate
1016095,1,2,2013-01-01,0,0,0,0,a,1,2013,1,1,1,1,1,1,0,1,0,0
1014980,1,3,2013-01-02,5530,668,1,0,0,1,2013,1,2,2,1,1,0,0,0,0,0
1013865,1,4,2013-01-03,4327,578,1,0,0,1,2013,1,3,3,1,1,0,0,0,0,0
1012750,1,5,2013-01-04,4486,619,1,0,0,1,2013,1,4,4,1,1,0,0,0,0,0
1011635,1,6,2013-01-05,4997,635,1,0,0,1,2013,1,5,5,1,1,0,0,0,0,0
1010520,1,7,2013-01-06,0,0,0,0,0,1,2013,1,6,6,1,1,0,0,0,0,0
1009405,1,1,2013-01-07,7176,785,1,1,0,1,2013,1,7,7,2,1,0,0,0,0,1
1008290,1,2,2013-01-08,5580,654,1,1,0,1,2013,1,8,8,2,1,0,0,0,0,0
1007175,1,3,2013-01-09,5471,626,1,1,0,1,2013,1,9,9,2,1,0,0,0,0,0
1006060,1,4,2013-01-10,4892,615,1,1,0,1,2013,1,10,10,2,1,0,0,0,0,0


# Examine STORE Data set more carefully

In [60]:
# count number of NaNS
store.isnull().sum()

Store                          0
StoreType                      0
Assortment                     0
CompetitionDistance            3
CompetitionOpenSinceMonth    354
CompetitionOpenSinceYear     354
Promo2                         0
Promo2SinceWeek              544
Promo2SinceYear              544
PromoInterval                544
dtype: int64

In [61]:
store.sum()

Store                         622170
CompetitionDistance          6010250
CompetitionOpenSinceMonth       5498
CompetitionOpenSinceYear     1528597
Promo2                           571
Promo2SinceWeek                13473
Promo2SinceYear              1148717
dtype: float64

## Need to determine different methods to deal with Nans

- http://stackoverflow.com/questions/27824954/how-to-handle-missing-nans-for-machine-learning-in-python
- https://www.quora.com/What-are-the-best-ways-to-account-for-missing-data-in-machine-learning
- https://www.kaggle.com/amhchiu/rossmann-store-sales/more-exploratory-data-analysis/notebook

## Store Type

In [63]:
store.head()

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,c,a,1270,9,2008,0,NaN,NaN,NaN
1,2,a,a,570,11,2007,1,13,2010,"Jan,Apr,Jul,Oct"
2,3,a,a,14130,12,2006,1,14,2011,"Jan,Apr,Jul,Oct"
3,4,c,c,620,9,2009,0,NaN,NaN,NaN
4,5,a,a,29910,4,2015,0,NaN,NaN,NaN


In [77]:
store.isnull().sum()

Store                          0
StoreType                      0
Assortment                     0
CompetitionDistance            3
CompetitionOpenSinceMonth    354
CompetitionOpenSinceYear     354
Promo2                         0
Promo2SinceWeek              544
Promo2SinceYear              544
PromoInterval                544
dtype: int64

In [83]:
pd.unique(store.StoreType.ravel())

array(['c', 'a', 'd', 'b'], dtype=object)

In [84]:
pd.unique(store.CompetitionOpenSinceMonth.ravel())

array([  9.,  11.,  12.,   4.,  10.,   8.,  nan,   3.,   6.,   5.,   1.,
         2.,   7.])

In [85]:
pd.unique(store.CompetitionOpenSinceYear.ravel())

array([ 2008.,  2007.,  2006.,  2009.,  2015.,  2013.,  2014.,  2000.,
        2011.,    nan,  2010.,  2005.,  1999.,  2003.,  2012.,  2004.,
        2002.,  1961.,  1995.,  2001.,  1990.,  1994.,  1900.,  1998.])

In [82]:
store.mean()

Store                         558.000000
CompetitionDistance          5404.901079
CompetitionOpenSinceMonth       7.224704
CompetitionOpenSinceYear     2008.668857
Promo2                          0.512108
Promo2SinceWeek                23.595447
Promo2SinceYear              2011.763573
dtype: float64

**Could set missing CompetitionDistance to mean (5405)**

In [86]:
# join the Train to the Store date frame
train_store = pd.merge(train, store, on='Store')
train_store.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,year,...,PromoFirstDate,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,2,2013-01-01,0,0,0,0,a,1,2013,...,0,c,a,1270,9,2008,0,NaN,NaN,NaN
1,1,3,2013-01-02,5530,668,1,0,0,1,2013,...,0,c,a,1270,9,2008,0,NaN,NaN,NaN
2,1,4,2013-01-03,4327,578,1,0,0,1,2013,...,0,c,a,1270,9,2008,0,NaN,NaN,NaN
3,1,5,2013-01-04,4486,619,1,0,0,1,2013,...,0,c,a,1270,9,2008,0,NaN,NaN,NaN
4,1,6,2013-01-05,4997,635,1,0,0,1,2013,...,0,c,a,1270,9,2008,0,NaN,NaN,NaN


In [87]:
train_store.shape

(886103, 29)

In [88]:
# count number of NaNS
train_store.isnull().sum()

Store                             0
DayOfWeek                         0
Date                              0
Sales                             0
Customers                         0
Open                              0
Promo                             0
StateHoliday                      0
SchoolHoliday                     0
year                              0
month                             0
day                               0
day_of_year                       0
weekofyear                        0
quarter                           0
is_month_start                    0
is_month_end                      0
is_quarter_start                  0
is_quarter_end                    0
PromoFirstDate                    0
StoreType                         0
Assortment                        0
CompetitionDistance            2326
CompetitionOpenSinceMonth    281569
CompetitionOpenSinceYear     281569
Promo2                            0
Promo2SinceWeek              439600
Promo2SinceYear             

In [89]:
train_store.sum()

PromoFirstDate                    68625
CompetitionDistance          4792453270
CompetitionOpenSinceMonth       4366776
CompetitionOpenSinceYear     1214317760
Promo2                           446503
Promo2SinceWeek                10432585
Promo2SinceYear               898255049
dtype: float64

In [90]:
train_store.loc[(train['Store'] == 674) &  (train_store['Open'] == 1)]

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,year,...,PromoFirstDate,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
673,1,6,2015-03-14,5225,599,1,0,0,0,2015,...,0,c,a,1270,9,2008,0,NaN,NaN,NaN
1788,3,3,2013-06-12,5520,657,1,0,0,0,2013,...,0,a,a,14130,12,2006,1,14,2011,"Jan,Apr,Jul,Oct"
4018,5,1,2015-06-15,7808,747,1,1,0,0,2015,...,1,a,a,29910,4,2015,0,NaN,NaN,NaN
5133,7,5,2013-09-13,9977,1118,1,1,0,0,2013,...,0,a,c,24000,4,2013,0,NaN,NaN,NaN
6248,8,2,2014-11-18,5834,766,1,0,0,0,2014,...,0,a,a,7520,10,2014,0,NaN,NaN,NaN
7363,10,6,2013-02-16,4340,511,1,0,0,0,2013,...,0,a,a,3160,9,2009,0,NaN,NaN,NaN
9593,12,4,2015-02-19,7904,936,1,1,0,0,2015,...,0,a,c,1070,NaN,NaN,1,13,2010,"Jan,Apr,Jul,Oct"
10708,14,3,2013-09-11,5831,666,1,1,0,0,2013,...,0,a,a,1300,3,2014,1,40,2011,"Jan,Apr,Jul,Oct"
12938,17,4,2013-02-14,5804,826,1,0,0,0,2013,...,0,a,a,50,12,2005,1,26,2010,"Jan,Apr,Jul,Oct"
14053,18,5,2013-12-13,7272,813,1,0,0,0,2013,...,0,d,c,13840,6,2010,1,14,2012,"Jan,Apr,Jul,Oct"
